In [1]:
!pip install fasthugs

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.4 requires wrapt~=1.12.1, but you have wrapt 1.14.1 which is incompatible.
tensorflow-transform 1.8.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
te

In [2]:
from transformers import AutoModelForSequenceClassification
from fastai.text.all import *
from fastai.callback.wandb import *

from fasthugs.learner import TransLearner
from fasthugs.data import TransformersTextBlock, TextGetter, get_splits, PreprocCategoryBlock

from datasets import load_dataset, concatenate_datasets

import random 
import numpy as np
import torch

def random_seed(seed_value): 
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    random.seed(seed_value) 


random_seed(42)

In [3]:
ds_name = 'glue'
model_name = "AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1"

max_len = 512
bs = 32
val_bs = bs*2

lr = 3e-5

In [4]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
def validate_task():
    assert task in GLUE_TASKS

In [5]:
from fastai.metrics import MatthewsCorrCoef, F1Score, PearsonCorrCoef, SpearmanCorrCoef

In [6]:
glue_metrics = {
    'cola':[MatthewsCorrCoef()],
    'sst2':[accuracy],
    'mrpc':[F1Score(), accuracy],
    'stsb':[PearsonCorrCoef(), SpearmanCorrCoef()],
    'qqp' :[F1Score(), accuracy],
    'mnli':[accuracy],
    'qnli':[accuracy],
    'rte' :[accuracy],
    'wnli':[accuracy],
}

In [7]:
task = 'mnli'
validate_task()

In [8]:
ds = load_dataset(ds_name, task)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
ds.keys()

dict_keys(['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

In [10]:
train_idx, valid_idx = get_splits(ds, valid='validation_matched')
valid_idx

(#9815) [392702,392703,392704,392705,392706,392707,392708,392709,392710,392711...]

In [11]:
train_ds = concatenate_datasets([ds['train'], ds['validation_matched']])

In [12]:
train_ds[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [13]:
lens = train_ds.map(lambda s: {'len': len(s['premise'])+len(s['hypothesis'])}, remove_columns=train_ds.column_names, num_proc=4, keep_in_memory=True)
train_lens = lens.select(train_idx)['len']
valid_lens = lens.select(valid_idx)['len']

#0:   0%|          | 0/100630 [00:00<?, ?ex/s]

#1:   0%|          | 0/100629 [00:00<?, ?ex/s]

#2:   0%|          | 0/100629 [00:00<?, ?ex/s]

#3:   0%|          | 0/100629 [00:00<?, ?ex/s]

In [14]:
dblock = DataBlock(blocks = [TransformersTextBlock(pretrained_model_name=model_name),
                             CategoryBlock(vocab={0:'entailment', 1:'neutral', 2:'contradiction'})],
                   get_x=TextGetter('premise', 'hypothesis'),
                   get_y=ItemGetter('label'),
                   splitter=IndexSplitter(valid_idx))

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [15]:
%%time
dl_kwargs=[{'res':train_lens}, {'val_res':valid_lens}]
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=val_bs, dl_kwargs=dl_kwargs, num_workers=4)

CPU times: user 1min 9s, sys: 1.87 s, total: 1min 11s
Wall time: 1min 15s


In [16]:
dls.show_batch(max_n=5)

,text,text_,category
0,well uh that's kind of obvious i mean they're even carrying it to to where now uh that they advertise on TV you know if your if you uh you know have done this or if you need this uh uh we'll sue for you and you don't have to pay us unless you but then what they don't tell you is that if you if they win you give them at least a third of the of the thing that they win so i don't know it is uh it's getting to be more business now rather than uh actually uh dealing with the crime than with uh um the uh punishment they the the lawyers are just in it for the money i'm i'm convinced i know i i agree with you i think you're real you're very right that the politicians should i think they,I think that there should be an equal representation of backgrounds in our politicians.,0
1,hum last month of the i took a i sponsored a youth group up there and uh it was funny we went to a uh oh we must have had thirty of us and we went into a Pizza Hut and you wander into a Pizza Hut with that kind of crowd and they sort of hate to see you coming so they're they get the business but with a crowd of people it's uh it it it was kind of funny and we went in first and then it was so crowded and we went and i'm trying to think it's right off anyway i don't know if it's Pizza Hut or Pizza Inn anyway we went to another one and and uh they the rates there are fairly reasonable but uh you know even those places are starting to have you know video games and things for kids,There was a big group of us that went to Pizza Hut.,0
2,my i had the same situation here my kids were we bought them up and put them in the public schools here in Rhode Island and they were not not too too bad and this was going back like ten years i guess ten years ago they were not too too bad at that time and i was fortunate enough to have the opportunity to move from Rhode Island to Texas with with TI with my job and i worked there in the southwest for ten almost ten or eleven years and the kids got a my kids got a much better education education in Texas simply because they had the opportunity to have more one on one,My kids had the chance to become better educated in Texas compared to Rhode island when I moved to Texas and worked for TI.,0
3,uh the bottom line that i after four years of working with the state prison system i've come to the conclusion if you're going to give the man the death sentence go ahead and fulfill the sentence understandably the Supreme Court says any time you hand down a death sentence to somebody they get a one appeal once that appeal fails within thirty days execute sentence if you're going to give the man the death sentence don't keep the guy on death row for eight nine ten years and make him worry about it,"If a man is sentenced to death, the state should carry out the sentence and put the man to death, there should be no appeal process, appeals are tyng up our court systems and costing too much.",1
4,"The Jenkins Committee (named after the chairman), a group formed by the AICPA in 1991 to address concerns over the relevance and usefulness of financial reporting, recommended in its 1994 report that standard setters develop a comprehensive reporting model that includes both financial information (financial statements and related disclosures) and nonfinancial information (such as high-level operating data and performance measures used by management, management's analysis of changes in financial and nonfinancial data, and forward-looking information about opportunities, risks, and management's plans).",The Jenkins Committee was formed in 1981.,2


In [17]:
import wandb

WANDB_NAME = f'{ds_name}-{task}-{model_name}'
GROUP = f'{ds_name}-{task}-{model_name}-{lr:.0e}'
NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'radam']

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
metrics = glue_metrics[task]
learn = TransLearner(dls, model, metrics=metrics).to_fp16()

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at AnonymousSub/rule_based_roberta_hier_triplet_epochs_1_shard_1 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_p

In [19]:
cbs = []
learn.fit_one_cycle(10, lr, cbs=cbs)

epoch,train_loss,valid_loss,accuracy,time
0,0.417000,0.395332,0.850942,39:33
1,0.409697,0.389799,0.852471,39:16
2,0.334324,0.396022,0.862252,39:19
3,0.270534,0.385005,0.870301,39:28
4,0.203109,0.454686,0.867142,39:33
5,0.143003,0.475764,0.869078,40:33
6,0.076785,0.573508,0.868059,41:06
7,0.065559,0.601458,0.867652,39:47
8,0.029959,0.653138,0.872236,39:39
9,0.021859,0.687144,0.871116,39:40


In [20]:
learn.show_results()

,text,text_,category,category_
0,yes they would they just wouldn't be able to own the kind of automobiles that they think they deserve to own or the kind of homes that we think we deserve to own we might have to you know just be able to i think if we a generation went without debt then the next generation like if if our our generation my husband and i we're twenty eight if we lived our lives and didn't become you know indebted like you know our generation before us that um the budget would balance and that we became accustomed to living with what we could afford which we wouldn't be destitute i mean we wouldn't be living on the street by any means but just compared to how spoiled we are we would be in our own minds but i feel like the generation after us would oh man it it,Society would be perfect and there would be no more war if we could just rid ourselves of our debt.,1,2
1,yes they would they just wouldn't be able to own the kind of automobiles that they think they deserve to own or the kind of homes that we think we deserve to own we might have to you know just be able to i think if we a generation went without debt then the next generation like if if our our generation my husband and i we're twenty eight if we lived our lives and didn't become you know indebted like you know our generation before us that um the budget would balance and that we became accustomed to living with what we could afford which we wouldn't be destitute i mean we wouldn't be living on the street by any means but just compared to how spoiled we are we would be in our own minds but i feel like the generation after us would oh man it it,Life will be great for subsequent generations if our generation goes without debt.,1,0
2,well the first thing for me is i wonder i see a couple of different ways of talking about what privacy is um if privacy is something that disturbs your private state i mean an invasion of privacy is something that disturbs your private state that's one thing and if privacy is something that comes into your private state and extracts information from it in other words finds something out about you that's another and the first kind of invasion of the first type of privacy seems invaded to me in very much everyday in this country but in the second type at least overtly uh where someone comes in and uh finds out information about you that should be private uh does not seem uh um obviously everyday,"Talking about privacy is a complicated topic, there are a couple different ways of talking about it, for example privacy is something that disturbs your private state...",0,0
3,"The rule prohibits the sale of nicotine-containing cigarettes and smokeless tobacco to individuals under the age of 18; requires manufacturers, distributors, and retailers to comply with various conditions regarding the sale and distribution of these products; requires retailers to verify a purchaser's age by photographic identification; prohibits all free samples; limits the distribution of these products through vending machines and self-service displays by permitting such methods of sale only in facilities where access by individuals under 18 is prohibited; limits the advertising and labeling to which children and adolescents are exposed; prohibits promotional, non-tobacco items such as hats and tee shirts; prohibits sponsorship of",This rule will make the sale of tobacco products to people under 18 years old legal in every state and Mexico.,2,2
4,"The rule prohibits the sale of nicotine-containing cigarettes and smokeless tobacco to individuals under the age of 18; requires manufacturers, distributors, and retailers to comply with various conditions regarding the sale and distribution of these products; requires retailers to verify a purchaser's age by photographic identification; prohibits all free samples; limits the distribution of these products through vending machines and self-service displays by permitting such methods of sale only in facilities where acc